3. Data Preparation & Feature Engineering

In [528]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from google.colab import files

In [529]:
df=pd.read_csv('https://raw.githubusercontent.com/mmm-pygame/Final-Project-Machine-Learning/refs/heads/main/data/splited/train.csv')

In [530]:
df.fillna({'Age':-1,'Name':"","Cabin":""},inplace=True)

Feature Engineering

In [531]:
df['FamilySize']=df['SibSp']+df['Parch']+1
df['IsAlone']=df['FamilySize'].apply(lambda x: 1 if x==1 else 0)

xử lý feature nhiễu và dữ liệu dạng categorical quan trọng

In [532]:
df['Missing Age Values']=df['Age'].apply(lambda x: 1 if x==-1 else 0)
df['Age']= df['Age'].apply(lambda x: float(x))
df['IsMale']=df['Sex'].apply(lambda x: 1 if x=='male' else 0)
df=df.drop(columns=['Sex','Name','PassengerId','Cabin','Ticket','Embarked'])

In [533]:
for col in df.columns:
  print(col," ",df[col].dtype," ",df[col].count()," ",len(df[col]))

Survived   int64   712   712
Pclass   int64   712   712
Age   float64   712   712
SibSp   int64   712   712
Parch   int64   712   712
Fare   float64   712   712
FamilySize   int64   712   712
IsAlone   int64   712   712
Missing Age Values   int64   712   712
IsMale   int64   712   712


Kiểm tra outliers

In [534]:
for col in df.columns:
  if ((np.issubdtype(df[col].dtype,np.int64)|np.issubdtype(df[col].dtype,np.float64) )and len(df[col].unique())>10):
    q1=float(df[col].quantile(0.25))
    q3=float(df[col].quantile(0.75))
    IQR=q3-q1
    lower_bound=q1-1.5*IQR
    upper_bound=q3+1.5*IQR
    transd=df[col].apply(lambda x: x if (x>=lower_bound) and (x<=upper_bound) else pd.NA)
    print(col," ",transd.dropna()," ",len(transd.dropna()))


Age   0      45.5
1      23.0
2      32.0
3      26.0
4       6.0
       ... 
707    21.0
708    -1.0
709    41.0
710    14.0
711    21.0
Name: Age, Length: 712, dtype: float64   712
Fare   0         28.5
1         13.0
2        7.925
3       7.8542
4       31.275
        ...   
704       26.0
706       46.9
707       7.65
708       31.0
709    14.1083
Name: Fare, Length: 616, dtype: object   616


Nhận xét: nếu xóa outliers sẽ mất lượng lớn dữ liệu=>Chọn cách dùng model mạnh như RandomForest để khắc phục và giữ outliers. Ngoài ra, kể cả tồn tại outliers nhưng với số lượng không quá áp đảo cũng sẽ ít ảnh hưởng hơn tới model accuracy
kể cả có là Linear Classifier.

In [535]:
df.head(890)

,Survived,Pclass,Age,SibSp,Parch,Fare,FamilySize,IsAlone,Missing Age Values,IsMale
0,0,1,45.5,0,0,28.5000,1,1,0,1
1,0,2,23.0,0,0,13.0000,1,1,0,1
2,0,3,32.0,0,0,7.9250,1,1,0,1
3,0,3,26.0,1,0,7.8542,2,0,0,1
4,0,3,6.0,4,2,31.2750,7,0,0,0
...,...,...,...,...,...,...,...,...,...,...
707,1,3,21.0,0,0,7.6500,1,1,0,0
708,0,1,-1.0,0,0,31.0000,1,1,1,1
709,0,3,41.0,2,0,14.1083,3,0,0,1
710,1,1,14.0,1,2,120.0000,4,0,0,0


In [536]:
for col in df.columns:
  print(col," ",df[col].dtype," ",df[col].count()," ",len(df[col]))

Survived   int64   712   712
Pclass   int64   712   712
Age   float64   712   712
SibSp   int64   712   712
Parch   int64   712   712
Fare   float64   712   712
FamilySize   int64   712   712
IsAlone   int64   712   712
Missing Age Values   int64   712   712
IsMale   int64   712   712


Giải quyết skewed distribution

In [537]:
skewedlist=df.drop(columns=['Missing Age Values']).skew().sort_values(ascending=False)
display(skewedlist)

,0
Fare,4.875066
SibSp,3.619385
FamilySize,2.753703
Parch,2.695459
Survived,0.511293
Age,0.200394
IsAlone,-0.419902
IsMale,-0.657700
Pclass,-0.677419


In [538]:
pskewed=skewedlist[skewedlist>0]
nskewed=skewedlist[skewedlist<0]
display(pskewed)
display(nskewed)

,0
Fare,4.875066
SibSp,3.619385
FamilySize,2.753703
Parch,2.695459
Survived,0.511293
Age,0.200394


,0
IsAlone,-0.419902
IsMale,-0.657700
Pclass,-0.677419


In [539]:
display(df)

,Survived,Pclass,Age,SibSp,Parch,Fare,FamilySize,IsAlone,Missing Age Values,IsMale
0,0,1,45.5,0,0,28.5000,1,1,0,1
1,0,2,23.0,0,0,13.0000,1,1,0,1
2,0,3,32.0,0,0,7.9250,1,1,0,1
3,0,3,26.0,1,0,7.8542,2,0,0,1
4,0,3,6.0,4,2,31.2750,7,0,0,0
...,...,...,...,...,...,...,...,...,...,...
707,1,3,21.0,0,0,7.6500,1,1,0,0
708,0,1,-1.0,0,0,31.0000,1,1,1,1
709,0,3,41.0,2,0,14.1083,3,0,0,1
710,1,1,14.0,1,2,120.0000,4,0,0,0


không transform hay scaler các categorical data

In [540]:
 trans=PowerTransformer(method='yeo-johnson')
 for x in df.drop(columns=['Age','Survived','Pclass','IsMale','Missing Age Values']).columns: df[x]=trans.fit_transform(df[[x]])

In [541]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df)

for x in df.drop(columns=['Survived','Pclass','IsMale','Missing Age Values']).columns: df[x]=trans.fit_transform(df[[x]])

In [542]:
display(df.drop(columns=['Missing Age Values']).skew().sort_values(ascending=False))
display(df)

,0
Parch,1.219072
SibSp,0.773655
Survived,0.511293
FamilySize,0.453525
Fare,-0.205956
IsAlone,-0.419902
Age,-0.433345
IsMale,-0.657700
Pclass,-0.677419


,Survived,Pclass,Age,SibSp,Parch,Fare,FamilySize,IsAlone,Missing Age Values,IsMale
0,0,1,1.105817,-0.686202,-0.562209,0.492841,-0.807746,0.812203,0,1
1,0,2,0.165721,-0.686202,-0.562209,-0.267652,-0.807746,0.812203,0,1
2,0,3,0.570404,-0.686202,-0.562209,-0.767327,-0.807746,0.812203,0,1
3,0,3,0.306148,1.434213,-0.562209,-0.776438,1.050607,-1.231219,0,1
4,0,3,-0.823448,1.523060,1.784125,0.579360,1.467474,-1.231219,0,0
...,...,...,...,...,...,...,...,...,...,...
707,1,3,0.068389,-0.686202,-0.562209,-0.803183,-0.807746,0.812203,0,0
708,0,1,-1.611398,-0.686202,-0.562209,0.571169,-0.807746,0.812203,1,1
709,0,3,0.934838,1.499503,-0.562209,-0.186098,1.298868,-1.231219,0,1
710,1,1,-0.302629,1.434213,1.784125,1.746812,1.387663,-1.231219,0,0


In [543]:
dff=df.to_csv('processed_train.csv',index=False,header=True)
display(pd.read_csv('processed_train.csv'))
files.download('processed_train.csv')

,Survived,Pclass,Age,SibSp,Parch,Fare,FamilySize,IsAlone,Missing Age Values,IsMale
0,0,1,1.105817,-0.686202,-0.562209,0.492841,-0.807746,0.812203,0,1
1,0,2,0.165721,-0.686202,-0.562209,-0.267652,-0.807746,0.812203,0,1
2,0,3,0.570404,-0.686202,-0.562209,-0.767327,-0.807746,0.812203,0,1
3,0,3,0.306148,1.434213,-0.562209,-0.776438,1.050607,-1.231219,0,1
4,0,3,-0.823448,1.523060,1.784125,0.579360,1.467474,-1.231219,0,0
...,...,...,...,...,...,...,...,...,...,...
707,1,3,0.068389,-0.686202,-0.562209,-0.803183,-0.807746,0.812203,0,0
708,0,1,-1.611398,-0.686202,-0.562209,0.571169,-0.807746,0.812203,1,1
709,0,3,0.934838,1.499503,-0.562209,-0.186098,1.298868,-1.231219,0,1
710,1,1,-0.302629,1.434213,1.784125,1.746812,1.387663,-1.231219,0,0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>